In [1]:
%run cv.ipynb

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


In [2]:
positive_twits = pd.read_csv('/Users/tyamgin/Downloads/positive.csv', sep=';', header=None).iloc[:,3].values

In [3]:
negative_twits = pd.read_csv('/Users/tyamgin/Downloads/negative.csv', sep=';', header=None).iloc[:,3].values

In [4]:
russian_stemmer = SnowballStemmer("russian")

def clear_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.)|(https?://)|(ok\.ru/))[^\s]+', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()
    
def stem_text(text):
    text = clear_text(text)
    return ' '.join(russian_stemmer.stem(word) for word in text.split() if word)
    
text = 'Василий https://some-link Геннадий @some_username Виталий ok.ru/group/343434'
assert stem_text(text) == 'васил геннад витал'


In [10]:
stemmed_positive = [stem_text(txt) for txt in positive_twits]
stemmed_negative = [stem_text(txt) for txt in negative_twits]

In [16]:
test_texts = parquet.read_table(input_path + '/texts/textsTest/').to_pandas().text.values
train_texts = parquet.read_table(input_path + '/texts/textsTrain').to_pandas().text.values

/usr/local/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [11]:
test_texts = []
stemmed_negative = []

In [17]:
stemmed_test = [stem_text(txt) for txt in test_texts]
print('test done')

test done


In [18]:
stemmed_train = [stem_text(txt) for txt in train_texts]
print('train done')

train done


In [ ]:
pd.DataFrame({'stems': stemmed_test}).to_pickle(output_path + '/stemmed_test')
pd.DataFrame({'stems': stemmed_train}).to_pickle(output_path + '/stemmed_train')
pd.DataFrame({'stems': stemmed_positive}).to_pickle(output_path + '/stemmed_positive')
pd.DataFrame({'stems': stemmed_negative}).to_pickle(output_path + '/stemmed_negative')

In [6]:
pd.read_pickle(output_path + '/stemmed_positive').stems.values

array(['хот я и школот но повер у нас то же сам d обществ профилир предмет тип',
       'да все так он немн похож на нег но мо мальчик все равн лучш d',
       'rt ну ты идиотк я испуга за теб', ...,
       'что происход со мно когд в эфир proactivefm звуч мо любим песн dctalk music',
       'любим я подар теб эт звезд им как звезд перевод подмышк',
       'посмотр непытайтесьпокинутьомск сегодн в вавилон в я там тож ест'],
      dtype=object)

In [11]:
vectorizer = TfidfVectorizer(ngram_range=(1,1), min_df=2)
matrix = vectorizer.fit_transform(#stemmed_positive + stemmed_negative + stemmed_test + stemmed_train)
    np.concatenate((pd.read_pickle(output_path + '/stemmed_positive').stems.values,
                    pd.read_pickle(output_path + '/stemmed_negative').stems.values,
                    pd.read_pickle(output_path + '/stemmed_test').stems.values,
                    pd.read_pickle(output_path + '/stemmed_train').stems.values)))
matrix

<4193784x667084 sparse matrix of type '<class 'numpy.float64'>'
	with 156437619 stored elements in Compressed Sparse Row format>

In [12]:
sparse.save_npz(output_path + "/pos_neg_sparse_1_2_.npz", matrix)

In [5]:
matrix = sparse.load_npz(output_path + "/pos_neg_sparse_2_2.npz")

In [6]:
matrix = matrix.astype(np.float32)
gc.collect()

127

In [7]:
%%time
svder = TruncatedSVD(n_components=80, n_iter=10)
feats = svder.fit_transform(matrix)
pd.DataFrame(feats[0:5])

CPU times: user 41min 53s, sys: 17min 27s, total: 59min 21s
Wall time: 1h 13min 12s


In [9]:
feats.shape

(4193784, 80)

In [11]:
matrix.shape

(4193784, 12061647)

In [10]:
positive_twits.shape[0] + negative_twits.shape[0] + 3410916 + 667957

4305707

In [14]:
3410916 + 667957 + positive_twits.shape[0]

4193784

In [15]:
ss = positive_twits.shape[0]
test_texts_shape_0 = parquet.read_table(input_path + '/texts/textsTest').to_pandas().shape[0]
pd.DataFrame({'embeddings': list(feats[ss:ss+test_texts_shape_0])}).to_pickle(output_path + '/test__80__svd')
pd.DataFrame({'embeddings': list(feats[ss+test_texts_shape_0:])}).to_pickle(output_path + '/train__80__svd')

In [17]:
feats.dtype

dtype('float32')

In [21]:
data = pd.DataFrame({'label': np.concatenate((np.repeat(1, positive_twits.shape[0]), 
                                              np.repeat(0, negative_twits.shape[0])))})
for i in range(feats.shape[1]):
    data['emb%d' % i] = feats[0:data.shape[0],i]

In [22]:
train, test = train_test_split(data, random_state=231, test_size=0.25)

In [29]:
model = LogisticRegression(random_state=444, solver='lbfgs', max_iter=1000).fit(train.drop('label', 1), train.label)

In [30]:
pred = model.predict_proba(test.drop('label', 1))[:,1]

In [72]:
accuracy_score(test.label, pred > 0.5) # 100

0.6518189352660072

In [79]:
accuracy_score(test.label, pred > 0.5) # 30

0.6127246116136769

In [89]:
accuracy_score(test.label, pred > 0.5) # 200

0.6731383025622035

In [31]:
accuracy_score(test.label, pred > 0.5) # 250 new

0.8093600663034086

In [34]:
model = LogisticRegression(random_state=444, solver='lbfgs', max_iter=1000).fit(data.drop('label', 1), data.label)

In [35]:
res_proba = model.predict_proba(feats[data.shape[0]:,:])[:,1]

In [36]:
stemmed_test_len = len(pd.read_pickle(output_path + '/stemmed_test').stems.values)
pd.DataFrame({'positivity': res_proba[0:stemmed_test_len]}).to_pickle(output_path + '/positivity_test2')
pd.DataFrame({'positivity': res_proba[stemmed_test_len:]}).to_pickle(output_path + '/positivity_train2')